In [ ]:
import requests
import json

# Define the base URL for the Kolada API
base_url = "http://api.kolada.se/v2/kpi"

# Make a GET request to the Kolada API to retrieve all KPIs
response = requests.get(base_url)
response.encoding = 'utf-8'

# Parse the JSON response
data = response.json()

# Print the structure of the data to understand its format
print("Data structure:")
print(json.dumps(data, ensure_ascii=False, indent=4))

# Assuming the data is a dictionary with a key 'values' that contains the list of KPIs
if isinstance(data, dict) and 'values' in data:
    kpi_entries = data['values']
else:
    kpi_entries = data

# Define the specific region (municipality) you want to filter on
region_name = "Stockholm"  # Replace with the specific region you want to filter on

# Filter the KPIs for those related to response time for ambulance, police, and fire department
keywords = ['responstid', 'ambulans', 'polis', 'brandkår']
response_time_kpis = [
    entry for entry in kpi_entries
    if any(keyword in entry.get('description', '').lower() for keyword in keywords)
]

# Further filter the KPIs based on the specific region
region_response_time_kpis = [
    entry for entry in response_time_kpis
    if region_name in entry.get('description', '').lower() or region_name in entry.get('title', '').lower()
]

# Print the filtered KPIs
print(f"KPIs related to response time for ambulance, police, and fire department in {region_name}: {len(region_response_time_kpis)}")
print(json.dumps(region_response_time_kpis, ensure_ascii=False, indent=4))

In [ ]:
import requests
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Define the base URL for the SOS Alarm API
base_url = "https://api.sosalarm.se/v1/incidents"  # Example endpoint, adjust as needed

# Define the parameters for the API request
params = {
    "year": "2023",        # Year for the data
    "region": "Stockholm"  # Region for the data
}

# Create a session with retry mechanism
session = requests.Session()
retry = Retry(
    total=5,  # Total number of retries
    backoff_factor=1,  # Wait 1, 2, 4, 8, 16 seconds between retries
    status_forcelist=[429, 500, 502, 503, 504],  # Retry on these status codes
)
adapter = HTTPAdapter(max_retries=retry)
session.mount("https://", adapter)

# Make a GET request to the SOS Alarm API
try:
    response = session.get(base_url, params=params)
    response.encoding = 'utf-8'

    # Check the status code of the response
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        print("Response text:")
        print(response.text)
    else:
        try:
            # Parse the JSON response
            data = response.json()

            # Print the structure of the data to understand its format
            print("Data structure:")
            print(json.dumps(data, ensure_ascii=False, indent=4))

            # Filter the data for specific metrics (e.g., response time)
            metrics = ['response_time', 'number_of_incidents']
            filtered_data = [
                entry for entry in data
                if any(metric in entry for metric in metrics)
            ]

            # Print the filtered data
            print(f"Filtered data: {len(filtered_data)}")
            print(json.dumps(filtered_data, ensure_ascii=False, indent=4))
        except json.JSONDecodeError as e:
            print("Error: Failed to parse JSON response")
            print("Response text:")
            print(response.text)
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

: 